# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [13]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
import openpyxl

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [14]:
stocks = pd.read_csv("https://raw.githubusercontent.com/datasets/s-and-p-500-companies-financials/master/data/constituents.csv")

stocks = pd.read_excel("./2_2_2021.xlsx")
stocks = stocks[["Ticker","Company","Industry"]]
stocks

,Ticker,Company,Industry
0,A,"Agilent Technologies, Inc.",Diagnostics & Research
1,AAL,American Airlines Group Inc.,Airlines
2,AAP,"Advance Auto Parts, Inc.",Specialty Retail
3,AAPL,Apple Inc.,Consumer Electronics
4,ABBV,AbbVie Inc.,Drug Manufacturers - General
...,...,...,...
500,YUM,"Yum! Brands, Inc.",Restaurants
501,ZBH,"Zimmer Biomet Holdings, Inc.",Medical Devices
502,ZBRA,Zebra Technologies Corporation,Communication Equipment
503,ZION,"Zions Bancorporation, National Association",Banks - Regional


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [15]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [16]:
symbol = "AAPL"

api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"

data = requests.get(api_url).json()

data




{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'RKG ABLAAEDQNL(GSCA EM/ETSTO )NLS',
 'calculationPrice': 'iexlasttrade',
 'open': None,
 'openTime': None,
 'openSource': 'aoilciff',
 'close': None,
 'closeTime': None,
 'closeSource': 'ioilafcf',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 121.29,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'March 4, 2021',
 'latestUpdate': 1652489382232,
 'latestVolume': None,
 'iexRealtimePrice': 125.21,
 'iexRealtimeSize': 104,
 'iexLastUpdated': 1691160986864,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 123.52,
 'previousVolume': 113395296,
 'change': -2.01,
 'changePercent': -0.01608,
 'volume': None,
 'iexMarketPercent': 0.01541552423294388,
 'iexVolum

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
price = data["latestPrice"]

market_cap = data["marketCap"]

price

121.29

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [18]:
my_columns = ["Ticker", "Stock Price","Market Cap", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns=my_columns)

In [19]:
final_dataframe.append(
    pd.Series(
        [symbol,
         price,
         market_cap,
         "N/A"
        ], index=my_columns),
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,AAPL,121.29,2084046030916,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [20]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol in stocks["Ticker"][:5]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data["latestPrice"],
                data["marketCap"],
                "N/A"
            ], index=my_columns),
        ignore_index=True
    )
    

In [21]:
stocks["Ticker"][:5]
print(any(stocks['Ticker'].str.contains('BF-B')))
#removing tickers not in data
stocks = stocks[stocks["Ticker"] != "BF-B"]
stocks = stocks[stocks["Ticker"] != "BRK-B"]


print(any(stocks['Ticker'].str.contains('BF-B')))

True
False


In [22]:
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,120.28,35797672416,N/A
1,AAL,22.11,13935470176,N/A
2,AAP,172.29,11360369385,N/A
3,AAPL,120.35,2106301854522,N/A
4,ABBV,106.49,194445642903,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [23]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [24]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

# find opdateret liste af stocks fra iexapis selv - get request?
# Fjern dårlig symbol fra liste?
symbol = "BF-B"
#api_call_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
#data = requests.get(api_call_url).json()
#print(data)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]["quote"]["latestPrice"],
                data[symbol]["quote"]["marketCap"],
                "N/A"
            ], 
            index = my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,120.080,35477291049,N/A
1,AAL,21.480,14127097771,N/A
2,AAP,169.600,11316519175,N/A
3,AAPL,124.640,2051984344585,N/A
4,ABBV,107.980,190415623493,N/A
...,...,...,...,...
498,YUM,106.930,31084833119,N/A
499,ZBH,156.290,33057673361,N/A
500,ZBRA,499.530,25707278354,N/A
501,ZION,54.005,9079233143,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [37]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except ValueError:
    print("Not a number \nPlease try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    try:
        val = float(portfolio_size)
    except:
        print("Dummy")

Enter the value of your portfolio:10000000


In [38]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,120.080,35477291049,165
1,AAL,21.480,14127097771,925
2,AAP,169.600,11316519175,117
3,AAPL,124.640,2051984344585,159
4,ABBV,107.980,190415623493,184
...,...,...,...,...
498,YUM,106.930,31084833119,185
499,ZBH,156.290,33057673361,127
500,ZBRA,499.530,25707278354,39
501,ZION,54.005,9079233143,368


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: